In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from agents.core.agent import core_agent

# from agents.feature.light_rag import insert_text
# insert_text('/Users/kishorkumar/AshaAi/chat-bot-backend/dataset')
chat_history = [
    {"role": "human", "content": "Hi there"},
    {"role": "ai", "content": "Hello! How can I help you?"},
    {"role": "human", "content": "Tell me about available jobs"},
    {"role": "ai", "content": "We have job openings in software and data science."}
]
new_query = """What specific career websites should we use for job data, and will HerKey provide a job
openings list?"""

inputs = {
    "user_query": new_query,
    "chat_history": chat_history,
    "tool_results": {},  # optional but safe to include
    "intent": "",        # will be set by extract_intent
    "response": "",      # will be set by generate_response
}
out = core_agent.invoke(inputs)


out

In [ ]:
out

In [ ]:
chat_history = [
    {"role": "human", "content": "Hi there"},
    {"role": "ai", "content": "Hello! How can I help you?"},
    {"role": "human", "content": "Tell me about available jobs"},
    {"role": "ai", "content": "We have job openings in software and data science."},
    {"role": "human", "content": "Can you check java job openings?"},
    {"role": "ai", "content": "I can help you find Java job openings.  To give you the best results, please tell me:\n\n* **Location:** (e.g., San Francisco, CA; New York; Remote)\n* **Experience level:** (e.g., Junior, Mid-level, Senior, Entry-level)* **Specific skills:** (e.g., Spring Boot, Java 8+, Microservices, Cloud Computing)\n* **Desired salary range (optional):**  This will help narrow down the results.\n\nOnce I have this information, I can search for relevant job openings and provide you with a list of matching jobs."},
]
new_query = "just check java job openings?"

inputs = {
    "user_query": new_query,
    "chat_history": [],
    "tool_results": {},  # optional but safe to include
    "intent": "",        # will be set by extract_intent
    "response": "",      # will be set by generate_response
}
for output in core_agent.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':")
        print(value)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

prompt = ChatPromptTemplate.from_messages([
        ("system", '''
        You are an intelligent assistant designed to classify user queries based on their intent. Always respond with only one of the following intents:

            1. feature: When the user is exploring features, offerings or general questions or seeks help on common topics (e.g., how-to, what is, where to find, troubleshooting).
            2. job: When the user asks related to jobs, events, or programs.
            3. profile_update: When the user wants to change or manage their personal profile information(ex: skill, experience, location etc.) or expresses interest in registering, creating an account, or signing up.
            4. unknown: When the user asks a question that does not fit into any of the above categories.

            Carefully analyze the user's query and return the most suitable intent as a single lowercase string (e.g., job).
        '''),
        # ("human", "{user_query}"),
    ])
prompt.append(HumanMessage(content="past content"))
prompt.format_messages(content="latest msg")

In [ ]:
from langchain.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region="in-en", max_results=20)
search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format="json")
print(search.run("site:indeed.com OR site:linkedin.com java jobs"))

In [ ]:
pip install firecrawl-py

In [ ]:
from firecrawl.firecrawl import FirecrawlApp

app = FirecrawlApp(api_key="fc-YOUR_API_KEY",
                   api_url="http//localhost:3002")

# Scrape a website:
scrape_status = app.scrape_url(
  'https://firecrawl.dev', 
  formats=["markdown", "html"]
)
print(scrape_status)

# Crawl a website:
crawl_status = app.crawl_url(
  'https://firecrawl.dev',
  limit=100,
  scrapeOptions={'formats': ['markdown', 'html']},
  poll_interval=30
)
print(crawl_status)

In [ ]:
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig

async def main():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://www.google.com/search?q=java+1+year+experience+job",
        )
        print(result.markdown[:300])  # Show the first 300 characters of extracted text

await main()

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from agents.wesearch_agent.web_search_agent import web_agent

query = "senior python backend engineer jobs in bengalore"
initial_state = {"input": query}

# Call the LangGraph agent
final_state = await web_agent.ainvoke(initial_state)

# Print the final output
print("🔍 Extracted Results:")
for idx, result in enumerate(final_state["output"]):
    print(f"\n--- Result {idx+1} ---\n{result}\n")

# query = "jobs in india"
# jobs = await run_wesearch(query)
# for job in jobs:
#     print(job)

In [ ]:
from web_scraping.crawler.herkey_scraper import scrape_jobs
jobs = await scrape_jobs()
for job in jobs:
    print(job)

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate

load_dotenv()

# Template for extracting job listings
prompt_template = """
You are given raw HTML content from a job listing website. Please extract the job title, company, and location and other information for each job posting.
return all the details in json format
HTML Content:
{html_content}

Job Title, Company, Location:
"""

prompt = PromptTemplate(input_variables=["html_content"], template=prompt_template)


from langchain_google_genai import ChatGoogleGenerativeAI

from utils.constants import MODEL_ID


model_id = MODEL_ID

llm = ChatGoogleGenerativeAI(model=model_id, temperature=0.0)

formated_prompt = prompt.format(html_content=jobs)
out = llm.invoke(formated_prompt)

In [ ]:
from scrapegraphai.graphs import SmartScraperGraph

# Define the configuration for the scraping pipeline
graph_config = {
    "llm": {
        "model": "ollama/gemma3:4b",
        "temperature": 0.0,
        "format": "json",
        "base_url": "http://localhost:11434",
        "model_tokens": 8192
    },
    "embedding": {
        "model": "ollama/nomic-embed-text",
        "base_url": "http://localhost:11434"
    },
    "verbose": True,
    "headless": False,
}

# Create the SmartScraperGraph instance
smart_scraper_graph = SmartScraperGraph(
    prompt="Extract useful information from the webpage, including a description of job, company, location, and other relevant details.",
    source="https://www.herkey.com/jobs",
    config=graph_config
)

# Run the pipeline
# result = await smart_scraper_graph.run()
import asyncio
import concurrent.futures

loop = asyncio.get_event_loop()

with concurrent.futures.ThreadPoolExecutor() as pool:
    result = await loop.run_in_executor(pool, smart_scraper_graph.run)


import json
print(json.dumps(result, indent=4))

In [ ]:
import chromadb
client = chromadb.HttpClient()
collection = client.create_collection("sample_collection")
# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=["This is document1", "This is document2"], # we embed for you, or bring your own
    metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on arbitrary metadata!
    ids=["doc1", "doc2"], # must be unique for each doc
)
results = collection.query(
    query_texts=["This is a query document"],
    n_results=2,
    where={"source": "google-docs"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

In [ ]:
results = collection.query(
    query_texts=["This is a query document"],
    n_results=2,
    where={"source": "google-docs"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)
results

In [ ]:
from agents.herkey.chromadb.db_connection import  insert_job_data

In [ ]:
insert_job_data()

In [ ]:
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient("mongodb://localhost:27017/")


In [ ]:
# Access the database
db = client["job_database"]


In [ ]:
from agents.herkey.tools.search_job import search_jobs_tool
# Example test
query = "developer"
location = "bangalore"
company = None
work_mode = None
min_experience = 1
max_experience = 3
min_salary = 6
max_salary = 12
skills = ["React"]
industry = ["technology"]

result = search_jobs_tool(query=query, location=location,
                        #   company=company, work_mode=work_mode, 
                        #    min_experience=min_experience,  )
                        #    min_salary=min_salary, max_salary=max_salary, 
                        skills=skills, industry=industry)

print(result)


In [ ]:
from agents.herkey.agents.planner_agent import app
# query = input("\nAsk something: ")
state = app.invoke({"query": "Is there any job openings in data science?"})
print("\n🔍 Final Answer:\n", state["final_answer"].content)

In [ ]:
from agents.feature.light_rag import insert_text
insert_text('/Users/kishorkumar/AshaAi/chat-bot-backend/dataset')

In [ ]:
from agents.feature.feature_agent import feature_agent
query = "Is there any job which can give me more than 10LPA? dont give me sources"  
feature_agent(query)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from browser_use import Agent
from dotenv import load_dotenv
from utils.constants import MODEL_ID

load_dotenv()



llm = ChatGoogleGenerativeAI(model=MODEL_ID, temperature=0.0)
async def main():
    agent = Agent(
        task="Naviagte to https://www.herkey.com/jobs and extract all the jobs data listed on the page, if you click on a job then you will get some more information releated to that job. Extract that info then go back then select another job, like that get all inforamtion for all the jobs",
        llm=llm,
    )
    result = await agent.run()
    print(result)

await main()

INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [telemetry] Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.
INFO     [agent] 🧠 Starting an agent with main_model=models/gemini-1.5-flash-8b +vision, planner_model=None, extraction_model=None 


/Users/kishorkumar/AshaAi/.venv/lib/python3.12/site-packages/browser_use/agent/service.py:321: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


INFO     [agent] 🚀 Starting task: Naviagte to https://www.herkey.com/jobs and extract all the jobs data listed on the page, if you click on a job then you will get some more information releated to that job. Extract that info then go back then select another job, like that get all inforamtion for all the jobs
INFO     [agent] 📍 Step 1


I0000 00:00:1745350193.240600  280325 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


INFO     [agent] 🤷 Eval: Unknown - Waiting for the page to load
INFO     [agent] 🧠 Memory: Navigated to https://www.herkey.com/jobs.  0 out of unknown number of jobs to extract.
INFO     [agent] 🎯 Next goal: Check if the page is loaded and extract the job listings.
INFO     [agent] 🛠️  Action 1/1: {"go_to_url":{"url":"https://www.herkey.com/jobs"}}
INFO     [controller] 🔗  Navigated to https://www.herkey.com/jobs
INFO     [agent] 📍 Step 2
INFO     [agent] 👍 Eval: Success - Extracted job data from the page
INFO     [agent] 🧠 Memory: Extracted job data from the page.  0 out of unknown number of jobs to extract remain.  Extracted job titles, companies, locations, experience levels, and skills for each job listing.  The data is stored in memory for further processing.
INFO     [agent] 🎯 Next goal: Process the extracted job data and decide on the next action.
INFO     [agent] 🛠️  Action 1/1: {"extract_content":{"goal":"extract all the job data listed on the page","should_strip_link_urls":fa

----------------------------------------------------------------------
➡️  Press [Enter] to resume or [Ctrl+C] again to exit... 